In [1]:
import sys
sys.version

'3.9.7 | packaged by conda-forge | (default, Sep 29 2021, 19:24:02) \n[Clang 11.1.0 ]'

In [2]:
import sqlite3

connection = sqlite3.connect('../data/movies.db')
cursor = connection.cursor()

In [4]:
records = cursor.execute('SELECT * FROM movies;')
for record in records:
    print(record)

('Pulp Fiction', 'Quentin Tarantino', 1994)
('Back to the Future', 'Steven Spielberg', 1985)
('Moonrise Kingdom', 'Wes Anderson', 2012)


In [6]:
cursor.execute('SELECT * FROM movies;')
cursor.fetchall()

[('Pulp Fiction', 'Quentin Tarantino', 1994),
 ('Back to the Future', 'Steven Spielberg', 1985),
 ('Moonrise Kingdom', 'Wes Anderson', 2012)]

In [8]:
cursor.execute('SELECT * FROM movies WHERE title LIKE "B%"')
cursor.fetchall()

[('Back to the Future', 'Steven Spielberg', 1985)]

In [37]:
import sqlalchemy as db
from pathlib import Path

db_path = Path(os.path.abspath('.')).parents[0].absolute().joinpath('data/movies.db')
engine = db.create_engine(f'sqlite:///{db_path}')
connection = engine.connect()

In [42]:
metadata = db.MetaData()
movies = db.Table('movies', metadata, autoload=True, autoload_with=engine)

In [43]:
metadata

MetaData()

In [44]:
movies

Table('movies', MetaData(), Column('title', TEXT(), table=<movies>), Column('director', TEXT(), table=<movies>), Column('year', INTEGER(), table=<movies>), schema=None)

In [45]:
query = db.select([movies])

In [48]:
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()

In [49]:
result_set

[('Pulp Fiction', 'Quentin Tarantino', 1994),
 ('Back to the Future', 'Steven Spielberg', 1985),
 ('Moonrise Kingdom', 'Wes Anderson', 2012)]

In [54]:
query = db.select([movies]).where(movies.columns.director == 'Steven Spielberg')
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
result_set

[('Back to the Future', 'Steven Spielberg', 1985)]

In [80]:
query = movies.insert().values(title='Psycho', director='Alfred Hitchcock', year=1960)
connection.execute(query)

In [81]:
query = db.select(movies)
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
result_set

[('Pulp Fiction', 'Quentin Tarantino', 1994),
 ('Back to the Future', 'Steven Spielberg', 1985),
 ('Moonrise Kingdom', 'Wes Anderson', 2012),
 ('Psycho', 'Alfred Hitchcock', 1960)]

In [82]:
movies.columns.director

Column('director', TEXT(), table=<movies>)

In [86]:
query = db.delete(movies).where(movies.columns.title == 'Psycho')
connection.execute(query)


In [87]:
query = db.select(movies)
result_proxy = connection.execute(query)
result_set = result_proxy.fetchall()
result_set

[('Pulp Fiction', 'Quentin Tarantino', 1994),
 ('Back to the Future', 'Steven Spielberg', 1985),
 ('Moonrise Kingdom', 'Wes Anderson', 2012)]

In [89]:
metadata.info

{}

In [6]:
import sqlalchemy as db

In [8]:
db.INTEGER

sqlalchemy.sql.sqltypes.INTEGER

In [12]:
from sqlalchemy import Column, Integer, String, Float, ForeignKey
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base

In [13]:
Base = declarative_base()

In [14]:
class Purchase(Base):
    __tablename__ = 'purchases'
    __table_args__ = {'schema': 'landon'}

    order_id = Column(Integer, primary_key=True)
    property_id = Column(Integer)
    property_city = Column(String(250))
    property_state = Column(String(250))
    product_id = Column(Integer)
    product_category = Column(String(250))
    product_name = Column(String(250))
    quantity = Column(Integer)
    product_price = Column(Float)
    order_total = Column(Float)


    def __repr__(self):
        return ''

In [22]:
purchase = Purchase()
purchase.__table__.columns.keys()

['order_id',
 'property_id',
 'property_city',
 'property_state',
 'product_id',
 'product_category',
 'product_name',
 'quantity',
 'product_price',
 'order_total']

In [26]:
s = 'Hello'
f's={repr(s)}'

"s='Hello'"

In [6]:
def format_class_repr_dict(cls_name, **attrs):
    s = ', '.join([f'{k}={repr(v)}' for k, v in attrs.items()])
    return s
    

In [7]:
format_class_repr_dict('Purchase', order_id=1, property_id=2, property_city="UP")


"order_id=1, property_id=2, property_city='UP'"

In [19]:
import psycopg2

conn = psycopg2.connect(host='localhost', user='postgres', password='', port='5432', database='red30')
conn.info

In [30]:
from functools import reduce

In [31]:
def get_column_names(conn, table):
    cur = conn.cursor()
    cur.execute(f"SELECT column_name FROM information_schema.columns WHERE table_schema='public' AND table_name='{table}'")
    cols = reduce(lambda x, y: x+y, cur.fetchall())
    return cols

In [32]:
cur = conn.cursor()
cur.execute('SELECT * FROM sales LIMIT 10;')

In [33]:
get_column_names(conn, 'sales')

('order_num',
 'order_type',
 'cust_name',
 'cust_state',
 'prod_category',
 'prod_number',
 'prod_name',
 'quantity',
 'price',
 'discount',
 'order_total')

In [22]:
from sqlalchemy import create_engine, MetaData, Table, func as sqlfunc
from sqlalchemy.orm import sessionmaker

Session = sessionmaker()
session = Session()

In [38]:
DATABASE = 'red30'
engine = create_engine(f'postgresql://postgres:@localhost:5432/{DATABASE}')

conn = engine.connect()


In [39]:
meta = MetaData(engine)

sales_table = Table('sales', meta, autoload=True, autoload_with=engine)


In [40]:
session.query(sales_table).count()

4977

In [41]:
sales_table.columns.keys()

['order_num',
 'order_type',
 'cust_name',
 'cust_state',
 'prod_category',
 'prod_number',
 'prod_name',
 'quantity',
 'price',
 'discount',
 'order_total']

In [45]:
row_count = session.query(sales_table).count()
last_row = sales_table.select().offset(row_count-1)
session.execute(last_row).first()

(1105909, 'Retail', 'Gussi Strodder', 'New York', 'Drone Kits', 'DK204', 'BYOD-300', 1, 89.0, 0.0, 89.0)

In [48]:
sales_table.c.order_num

Column('order_num', INTEGER(), table=<sales>, primary_key=True, nullable=False)

In [49]:
session.query(sqlfunc.max(sales_table.c.order_num)).scalar()

1105910

In [57]:
delete_statement = sales_table.delete().where(sales_table.c.cust_name == 'Akshay Bhatia')
session.execute(delete_statement)
session.commit()

In [59]:
import psycopg2
DATABASE = 'red30'

conn = psycopg2.connect(
    database=DATABASE,
    host='localhost',
    user='postgres',
    password='',
    port='5432')

In [62]:
cur = conn.cursor()
cur.execute('SELECT max(order_num) FROM sales')
cur.fetchone()[0]

1105911

In [63]:
conn.autocommit

False